# Light the Blinkt! and Read the AS7341

Welcome to the beginning of a series of tutorials on using the self-driving lab demo!
This notebook will show you how to use the Blinkt! and AS7341 to light up the LEDs and
print out the light sensor data.

## Resources

This tutorial is based on extracting and modifying snippets from the following two resources:

- [Getting Started with Blinkt!](https://learn.pimoroni.com/article/getting-started-with-blinkt)
- [README for CircuitPython AS7341](https://github.com/adafruit/Adafruit_CircuitPython_AS7341)

## Setup

First, we'll import some items from the Blinkt! and AS7341 Python libraries. If you're
working on a Raspberry Pi, you might run into new issues with installing even basic
packages such as NumPy. The following are some resources I referred to while troubleshooting:

- [How do I install pandas on Raspberry Pi? [Raspberry Pi Stack Exchange]](https://raspberrypi.stackexchange.com/a/108041/137101)
- [Importing the numpy c-extensions failed on raspberrypi [NumPy GitHub]](https://github.com/numpy/numpy/issues/16012#issuecomment-615927988)

However, I eventually found that setting up my Raspberry Pi 400 with a 64-bit Linux
operating system (`Ubuntu 22.04.1 LTS`) via [`rpi-imager`](https://www.raspberrypi.com/software/), and then installing
[Mambaforge](https://github.com/conda-forge/miniforge) resolved many headaches. Ensure
that you're using 64-bit RPi Linux OS and 64-bit Mambaforge installer.

After installing the correct operating system, you can follow the docs linked above to install Mambaforge you can run:
```bash
wget "https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-$(uname)-$(uname -m).sh"
bash Mambaforge-$(uname)-$(uname -m).sh
```

### Imports

We'll start out with some basic imports from the `blinkt` and `adafruit_as7341` libraries.

In [1]:
from blinkt import set_pixel, set_brightness, show, clear
import blinkt
from time import sleep
import board
from adafruit_as7341 import AS7341
import pandas as pd

### Blinkt!

Let's set the brightness to a relatively low value and set the pixel in
the 0-th position to the RGB values of (255, 255, 255) of white. Nothing happened
though! That's because we need to use the `show()` command to show our settings.

In [2]:
set_brightness(0.1)
set_pixel(0, 255, 255, 255)

Ok, the LED turns on now via the `show()` command. How do we turn it off?

In [3]:
show()

We turn off the LED by using the `clear()` command followed by `show()` again.

In [4]:
clear()
show()

### AS7341

For the sensor, setting up `I2C` helps you transfer data along the physical wires, and
setting up the `AS7341` class allows you to send high-level commands to the sensor.
Finally, the bar_graph lets you see the measured values in real time.

In [5]:
i2c = board.I2C()  # uses board.SCL and board.SDA
sensor = AS7341(i2c)

## Experiment

We'll start out with the pixel set to white, and then move through a loop 20 times. When
`i==3`, we'll set the LED to red. When `i==7`, we'll set the LED to green. When `i==11`, we'll set the LED to blue.
When `i==15`, we'll set the LED to white again all the way until the last iteration,
`i==19`. At each iteration, we'll call the `show()` command again, pause briefly, and
then read the sensor data using the `all_channels` property. While not shown here,
individual channels can be accessed via their name, e.g. `channel_415nm`. There are two
other channels, `channel_clear` and `channel_nir` (nir $\equiv$ near-infrared), which we
will ignore. Finally, there is flicker detection supported at two frequencies, and we'll
also ignore this.

In [8]:
clear()
set_pixel(0, 255, 255, 255) # white
blinkt.show()

intensities = []

for i in range(20):
    if i == 3:
        set_pixel(0, 255, 0, 0) # red

    if i == 7:
        set_pixel(0, 0, 255, 0) # green

    if i == 11:
        set_pixel(0, 0, 0, 255) # blue
    
    if i == 15:
        set_pixel(0, 255, 255, 255) # white

    show()

    sleep(0.2)

    data = sensor.all_channels
    print(data)
    intensities.append(data)

blinkt.clear()
blinkt.show()


(170, 2208, 6362, 3145, 1580, 820, 3158, 452)
(170, 2205, 6361, 3144, 1579, 818, 3154, 453)
(170, 2204, 6360, 3143, 1580, 817, 3152, 453)
(170, 2203, 6359, 3141, 1580, 816, 3150, 453)
(51, 67, 86, 70, 60, 565, 2978, 143)
(51, 67, 86, 70, 60, 566, 2979, 143)
(51, 67, 86, 70, 60, 566, 2981, 143)
(51, 68, 86, 70, 60, 567, 2981, 143)
(70, 52, 371, 2891, 1455, 219, 161, 153)
(70, 52, 371, 2890, 1454, 219, 161, 153)
(70, 52, 371, 2891, 1455, 219, 161, 153)
(70, 52, 371, 2890, 1455, 219, 161, 153)
(53, 2111, 5961, 234, 111, 89, 111, 200)
(53, 2111, 5960, 234, 111, 89, 110, 200)
(53, 2110, 5961, 234, 111, 89, 110, 200)
(53, 2110, 5963, 234, 111, 89, 111, 200)
(170, 2206, 6362, 3143, 1580, 816, 3151, 453)
(170, 2205, 6363, 3140, 1577, 812, 3145, 450)
(169, 2202, 6361, 3138, 1575, 808, 3137, 447)
(169, 2201, 6361, 3136, 1576, 808, 3137, 448)


## Visualization

We'll plot the channel vs. time step with intensity given by the color. Also, we'll
annotate each of the regions corresponding to different `set_pixel()` settings, namely
"white" `(255,255,255)`, "red" `(255,0,0)`, "green" `(0,255,0)`, "blue" `(0,0,255)`, and
"white" again.

In [43]:
import numpy as np
import plotly.express as px

fig = px.imshow(np.array(intensities).T)
fig.update_layout(
    xaxis=dict(title="time_step", tickvals=[0, 3, 7, 11, 15, 19]),
    yaxis=dict(
        title="wavelength (nm)",
        tickvals=list(range(8)),
        ticktext=[
            "415 (Violet)",
            "445 (Indigo)",
            "480 (Blue)",
            "515 (Cyan)",
            "560 (Green)",
            "615 (Yellow)",
            "670 (Orange)",
            "720 (Red)",
        ],
    ),
    coloraxis_colorbar=dict(title="intensity"),
)
[
    fig.add_annotation(
        text=txt, xref="paper", yref="paper", x=x, y=0.95, showarrow=False
    )
    for x, txt in zip(
        [0.075, 0.275, 0.5, 0.725, 0.93], ["white", "red", "green", "blue", "white"]
    )
]
pass # to avoid output from above lines

Note that the colorscale colors don't correspond to the colors of the
individual wavelengths; they just correspond to the intensities. You can see
qualitative differences in the spectra at each of the distinct regions ("white", "red", "green",
"blue", and "white"). Also notice that there's some mismatch between what the
sensor refers to as "red" and the "red" in the LED.

In [45]:
fig.show()

### Example Visualization

![intensities](intensities.png)

## Up Next

[`2.0-random-search.ipynb`](2.0-random-search.ipynb)

> 🚗 Let's run a test drive of 100 random search iterations!

## Code Graveyard

In [ ]:
# def bar_graph(read_value):
#     scaled = int(read_value / 1000)
#     return "[%5d] " % read_value + (scaled * "*")



# print("F1 - 415nm/Violet  %s" % bar_graph(sensor.channel_415nm))
# print("F2 - 445nm//Indigo %s" % bar_graph(sensor.channel_445nm))
# print("F3 - 480nm//Blue   %s" % bar_graph(sensor.channel_480nm))
# print("F4 - 515nm//Cyan   %s" % bar_graph(sensor.channel_515nm))
# print("F5 - 555nm/Green   %s" % bar_graph(sensor.channel_555nm))
# print("F6 - 590nm/Yellow  %s" % bar_graph(sensor.channel_590nm))
# print("F7 - 630nm/Orange  %s" % bar_graph(sensor.channel_630nm))
# print("F8 - 680nm/Red     %s" % bar_graph(sensor.channel_680nm))
# print("Clear              %s" % bar_graph(sensor.channel_clear))
# print("Near-IR (NIR)      %s" % bar_graph(sensor.channel_nir))
# print("\n------------------------------------------------")